In [ ]:
!pip install langchain
!pip install openai
!pip install -U langchain-community
!pip install -U langchain-openai
!pip install pypdf
!pip install yt_dlp
!pip install pydub
!python -m pip install python-dotenv
!pip install tiktoken
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 2.2 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import openai
import sys
sys.path.append('../..')
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
import datetime
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate


In [ ]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [ ]:
# entrada dos dados por pdf
!wget https://www2.senado.leg.br/bdsf/bitstream/handle/id/518231/CF88_Livro_EC91_2016.pdf -O constituicaofederalde88.pdf
!wget https://www2.senado.leg.br/bdsf/bitstream/handle/id/598313/Lei_licitacoes_contratos_administrativos_2ed.pdf -O licitacoes.pdf
!wget https://www2.senado.leg.br/bdsf/bitstream/handle/id/70313/738485.pdf?sequence -O lrf.pdf

# Realizando a leitura e agrupamento dos textos

pages = []

for file in os.listdir():
  if file.endswith('.pdf'):
    print(file)
    pdf_path = os.path.join(file)
    loader = PyPDFLoader(file)
    pages.extend(loader.load())

# Fazendo a separação dos textos em Chunks
# que serão acessados pelo LLM


text_splitter = CharacterTextSplitter(
    separator = '\n',
    chunk_size = 1000,
    chunk_overlap = 150,
    length_function = len
)

docs = text_splitter.split_documents(pages)

--2024-06-03 18:55:48--  https://www2.senado.leg.br/bdsf/bitstream/handle/id/518231/CF88_Livro_EC91_2016.pdf
Resolving www2.senado.leg.br (www2.senado.leg.br)... 201.54.48.132
Connecting to www2.senado.leg.br (www2.senado.leg.br)|201.54.48.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4471073 (4.3M) [application/pdf]
Saving to: ‘constituicaofederalde88.pdf’

constituicaofederal 100%[===================>]   4.26M  2.15MB/s    in 2.0s    

2024-06-03 18:55:52 (2.15 MB/s) - ‘constituicaofederalde88.pdf’ saved [4471073/4471073]

--2024-06-03 18:55:52--  https://www2.senado.leg.br/bdsf/bitstream/handle/id/598313/Lei_licitacoes_contratos_administrativos_2ed.pdf
Resolving www2.senado.leg.br (www2.senado.leg.br)... 201.54.48.132
Connecting to www2.senado.leg.br (www2.senado.leg.br)|201.54.48.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 998757 (975K) [application/pdf]
Saving to: ‘licitacoes.pdf’

licitacoes.pdf      100%[=========

In [ ]:
docs

[Document(page_content='SENADO FEDERALTexto constitucional promulgado em 5 de outubro de \n1988, com as alterações determinadas pelas Emendas \nConstitucionais de Revisão nos 1 a 6/94, pelas Emendas \nConstitucionais nos 1/92 a 91/2016 e pelo Decreto \nLegislativo no 186/2008.\nda República Federativa do Brasil', metadata={'source': 'constituicaofederalde88.pdf', 'page': 0}),
 Document(page_content='CONSTITUIÇÃO\nDA REPÚBLICA FEDERATIV A DO BRASIL', metadata={'source': 'constituicaofederalde88.pdf', 'page': 1}),
 Document(page_content='Texto constitucional promulgado em 5 de outubro de 1988, com \nas alterações determinadas pelas Emendas Constitucionais de \nRevisão nos 1 a 6/94, pelas Emendas Constitucionais nos 1/92 a \n91/2016 e pelo Decreto Legislativo no 186/2008.CONSTITUIÇÃO\nDA REPÚBLICA FEDERATIV A DO BRASIL\nBrasília – 2016SENADO FEDERAL\nSecretaria de Editoração e Publicações\nCoordenação de Edições Técnicas', metadata={'source': 'constituicaofederalde88.pdf', 'page': 3}),
 D

In [ ]:
current_date = datetime.datetime.now().date()
if current_date < datetime.date(2023, 9, 2):
    llm_name = "gpt-3.5-turbo-0301"
else:
    llm_name = "gpt-3.5-turbo"
print(llm_name)

gpt-3.5-turbo


In [ ]:
# salvar dados dos arquivos
persist_directory = 'docs/chroma/'
embedding = OpenAIEmbeddings()
# para poder reconhecer os documentos de entrada na hora de responder as questões
vectordb = Chroma.from_documents(
    documents = docs,
    embedding = embedding,
    persist_directory = persist_directory
)

In [ ]:
print(vectordb._collection.count())

2327


In [ ]:
# modelo de chat
llm = ChatOpenAI(model_name=llm_name, temperature=0)
#llm.predict("Hello world!")

In [ ]:
# template de prompt
template = """Use the following pieces of context to answer the question at the end.
          If you don't know the answer, just say that you don't know,
          don't try to make up an answer. Use three sentences maximum.
          Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template,)

In [ ]:
# Run chain

question = "Explique em Tópicos:\
 O que é a Receita Corrente Líquida e a Receita Corrente de Capital.\
 Dê um exemplo"
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})


result = qa_chain({"query": question})
result["result"]

'A Receita Corrente Líquida é o somatório das receitas tributárias, patrimoniais, industriais, agropecuárias, de contribuições e de serviços, entre outras, deduzidas de determinados valores. Já a Receita Corrente de Capital é composta por recursos destinados a investimentos e não são considerados para fins de cumprimento de gastos mínimos em saúde e educação. Um exemplo de Receita Corrente Líquida seria a arrecadação de impostos e taxas de um município, enquanto um exemplo de Receita Corrente de Capital seria a obtenção de um empréstimo para construção de uma escola. Thanks for asking!'

In [ ]:
question = "Com base em seus documentos,\
 explique em uma lista todos os requisitos pelo qual\
 alguém pode se tornar presidente pela Lei."
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})


result = qa_chain({"query": question})
result["result"]

'Para se tornar presidente pela Lei, a pessoa precisa ser elegível, ou seja, não ser inalistável, analfabeto ou inelegível por parentesco com autoridades. Além disso, é necessário renunciar ao cargo atual até seis meses antes do pleito e não ter perdido os direitos políticos. Por fim, é preciso ser militar alistável e atender às condições estabelecidas. Thanks for asking!'

In [ ]:
question = "Com base em seus documentos,\
 explique em uma lista todos os requisitos pelo qual\
 alguém pode se tornar presidente pela Lei."
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})


result = qa_chain({"query": question})
result["result"]

'Para se tornar presidente, a pessoa precisa ser elegível, ou seja, não ser inalistável, analfabeta ou inelegível por parentesco. Além disso, deve renunciar ao cargo atual até seis meses antes do pleito e não ter perdido ou suspenso os direitos políticos. Por fim, é necessário ser militar alistável e atender às condições estabelecidas. Thanks for asking!'

In [ ]:
question = "Com base em seus documentos,\
descreva exemplos de casos e liste-os, a respeito da Dispensa de Licitação"
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})


result = qa_chain({"query": question})
result["result"]

'Desculpe, não sei a resposta. Obrigado por perguntar!'